In [1]:
import pandas as pd
import numpy as np
from itertools import combinations, product

In [2]:
# read in manually updated csvs
drivers = pd.read_csv('f1 - drivers.csv')
constructors = pd.read_csv('f1 - constructors.csv')

In [4]:
# points per position: how many points does each race position score, on average?
ppp = drivers.sort_values('Season Points per Race',ascending=False)['Season Points per Race']

In [5]:
# expected finish order
efo = drivers.sort_values(['Implied Win Probability','Season Points per Race'],ascending=False)[['Driver','Season Points per Race','Cost','Constructor']]

In [6]:
# expected points (weighting is untested)
efo['Avg. Points per Position'] = list(ppp)
efo['Expected Points'] = 0.75*efo['Avg. Points per Position'] + 0.25*efo['Season Points per Race']
drivers = efo

In [7]:
# list every combination of 5 drivers 'dri'
dri = []
for driver_combo in combinations(drivers['Driver'].values,5):
    dri.append(driver_combo)

In [9]:
# list every combination of 2 constructors 'con'
con = []
for con_combo in combinations(constructors['Team'].values,2):
    con.append(con_combo)

In [10]:
# list every combination of those dri and con 'lineup'
lineup = []
for lineup_product in product(dri,con):
    lineup.append(lineup_product)

In [11]:
# pull the lineups into a dataframe
rows = []
for entry in lineup:
    d1 = entry[0][0]
    d2 = entry[0][1]
    d3 = entry[0][2]
    d4 = entry[0][3]
    d5 = entry[0][4]
    c1 = entry[1][0]
    c2 = entry[1][1]
    row = [d1,d2,d3,d4,d5,c1,c2]
    rows.append(row)
    
cols = ['D1','D2','D3','D4','D5','C1','C2']
df = pd.DataFrame(rows,columns=cols)

In [13]:
# map costs back onto dataframe
driver_map = drivers[['Driver','Cost']].set_index('Driver')['Cost'].to_dict()

for col in ['D1','D2','D3','D4','D5']:
    new_col_str = col + '_budget'
    df[new_col_str] = df[col].map(driver_map)

con_map = constructors[['Team','Cost']].set_index('Team')['Cost'].to_dict()

for col in ['C1','C2']:
    new_col_str = col + '_budget'
    df[new_col_str] = df[col].map(con_map)

df['total_budget'] = df[['D1_budget','D2_budget','D3_budget','D4_budget','D5_budget','C1_budget','C2_budget']].sum(axis=1)

In [14]:
# sum constructor expected points from its drivers
constructors['Expected Points'] = constructors['Team'].map(drivers.groupby('Constructor')['Expected Points'].sum())

In [15]:
# map win probabilities back onto dataframe
driver_map = drivers[['Driver','Expected Points']].set_index('Driver')['Expected Points'].to_dict()

for col in ['D1','D2','D3','D4','D5']:
    new_col_str = col + '_xp'
    df[new_col_str] = df[col].map(driver_map)

con_map = constructors[['Team','Expected Points']].set_index('Team')['Expected Points'].to_dict()

for col in ['C1','C2']:
    new_col_str = col + '_xp'
    df[new_col_str] = df[col].map(con_map)

# the top driver you assign your 'DRS Boost' to gets their score doubled
df['D1_xp'] = df['D1_xp']*2

df['total_xp'] = df[['D1_xp','D2_xp','D3_xp','D4_xp','D5_xp','C1_xp','C2_xp']].sum(axis=1)

In [16]:
# reset index
df = df.reset_index()

In [17]:
# list the remaining lineups by combined expected points
drop = ['D1_xp', 'D2_xp', 'D3_xp', 'D4_xp','D5_xp', 'C1_xp', 'C2_xp','D1_budget','D2_budget', 'D3_budget', 'D4_budget', 'D5_budget', 'C1_budget','C2_budget']
df.drop(columns=drop,inplace=True)

In [18]:
# sort by winningest
df.sort_values(['total_xp','total_budget'],ascending=[False,True],inplace=True)

In [19]:
def best_available_lineups(df,team:list,budget:float,free_transfers:int):
    '''
    given a dataframe of upcoming race probs and your team info, spits out the best lineups to switch to
    
    df: pandas dataframe of all possible lineups, with corresponding total_budget and total_wp
    team_n: a list of the team's current entrants
    budget: the team's current cost cap
    free_transfers: the number of free transfers available
    
    '''
    df = df[df['total_budget'] <= budget] # filter by budget

    # subset to teams that share enough entrants with the current entry
    # to stay under the free transfer limit
    t1 = set(team)
    match_indices = [] # empty list to store match indices
    for possible_entry in df.values:
        t2 = set(possible_entry)
        if len(t1&t2) >= (7 - free_transfers):
            match_indices.append(possible_entry[0])
    df = df[df['index'].isin(match_indices)]
    return df

In [20]:
# team 1
team_1 = ['Max Verstappen','Ollie Bearman','Carlos Sainz','Fernando Alonso','Isack Hadjar','McLaren','Williams']
t1_new = best_available_lineups(df,team_1,budget=111.1,free_transfers=2)
t1_new.head()

,index,D1,D2,D3,D4,D5,C1,C2,total_budget,total_xp
142881,142881,Max Verstappen,Isack Hadjar,Fernando Alonso,Yuki Tsunoda,Carlos Sainz,McLaren,Racing Bulls,108.3,148.55
143103,143103,Max Verstappen,Isack Hadjar,Fernando Alonso,Liam Lawson,Nico Hulkenberg,McLaren,Williams,110.0,147.55
143556,143556,Max Verstappen,Isack Hadjar,Fernando Alonso,Nico Hulkenberg,Carlos Sainz,McLaren,Racing Bulls,105.5,147.50
129653,129653,Max Verstappen,Kimi Antonelli,Isack Hadjar,Fernando Alonso,Carlos Sainz,McLaren,Kick Sauber,110.3,147.35
143241,143241,Max Verstappen,Isack Hadjar,Fernando Alonso,Liam Lawson,Carlos Sainz,McLaren,Racing Bulls,105.2,147.05


In [21]:
# team 2
team_2 = ['Oscar Piastri','Pierre Gasly','Carlos Sainz','Fernando Alonso','Gabriel Bortoleto','McLaren','Ferrari']
t2_new = best_available_lineups(df,team_2,budget=117.5,free_transfers=2)
t2_new.head()

,index,D1,D2,D3,D4,D5,C1,C2,total_budget,total_xp
388981,388981,Oscar Piastri,Isack Hadjar,Fernando Alonso,Gabriel Bortoleto,Pierre Gasly,McLaren,Red Bull,115.3,162.250
389161,389161,Oscar Piastri,Isack Hadjar,Fernando Alonso,Carlos Sainz,Pierre Gasly,McLaren,Red Bull,115.5,162.150
388485,388485,Oscar Piastri,Isack Hadjar,Fernando Alonso,Nico Hulkenberg,Pierre Gasly,McLaren,Ferrari,117.3,162.100
388170,388170,Oscar Piastri,Isack Hadjar,Fernando Alonso,Liam Lawson,Pierre Gasly,McLaren,Ferrari,117.0,161.650
388847,388847,Oscar Piastri,Isack Hadjar,Fernando Alonso,Gabriel Bortoleto,Carlos Sainz,McLaren,Mercedes,115.0,161.075


In [22]:
# team 3
team_3 = ['Lando Norris','Liam Lawson','Carlos Sainz','Fernando Alonso','Isack Hadjar','McLaren','Racing Bulls']
t3_new = best_available_lineups(df,team_3,budget=109.7,free_transfers=2)
t3_new.head()

,index,D1,D2,D3,D4,D5,C1,C2,total_budget,total_xp
142701,142701,Max Verstappen,Isack Hadjar,Fernando Alonso,Yuki Tsunoda,Liam Lawson,McLaren,Racing Bulls,108.5,151.025
143106,143106,Max Verstappen,Isack Hadjar,Fernando Alonso,Liam Lawson,Nico Hulkenberg,McLaren,Racing Bulls,105.7,149.975
280806,280806,Lando Norris,Isack Hadjar,Fernando Alonso,Liam Lawson,Nico Hulkenberg,McLaren,Racing Bulls,107.1,149.825
281166,281166,Lando Norris,Isack Hadjar,Fernando Alonso,Nico Hulkenberg,Oliver Bearman,McLaren,Racing Bulls,107.7,149.150
143151,143151,Max Verstappen,Isack Hadjar,Fernando Alonso,Liam Lawson,Oliver Bearman,McLaren,Racing Bulls,106.0,148.850
